#Set up Colab and Drive for Project

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [0]:
%cd '/content/gdrive/My Drive/MVP Datensatz/'
train_dir = 'MVP Datensatz/Trainingsdatensatz5'

In [2]:
from keras import optimizers
from keras import layers
from keras import models
from keras import initializers

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import backend as K


Using TensorFlow backend.


#Global Variables


In [0]:
batch_s = 30  # global batch_size
img_width = 299
img_height = 299
opt = optimizers.Adam(learning_rate=0.0009)
epochs = 20
Abfragebild = 'Testbilder/P1040695.JPG'

Speicherort_KI_Modell = '/content/gdrive/My Drive/Keras Models'
Name_KI_Modell = '/Xception_5-1.h5'


#Create Base Model, Training and Valdidation Generators


In [0]:
# Xception

from keras.applications import Xception
base_model = Xception(
  weights = 'imagenet',
  include_top = False,
  input_shape = (299,299,3) 
 )

base_model.summary()


In [0]:
train_datagen = ImageDataGenerator(
                          featurewise_center = False,
                          samplewise_center = False,
                          featurewise_std_normalization = False,
                          samplewise_std_normalization = False,
                          rotation_range = 359,
                          width_shift_range = 30.0,
                          height_shift_range = 0.0,
                          brightness_range = None,
                          channel_shift_range = 50.0,
                          horizontal_flip = True, 
                          vertical_flip = True, 
                          rescale = 1./255,
                          validation_split = 0.2, 
                          interpolation_order = 1, 
                          dtype = 'float32'
                          )


Found 113 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [0]:
valid_datagen = ImageDataGenerator(
                          rescale=1./255,
                          validation_split=0.2
                          )


In [0]:
train_generator = train_datagen.flow_from_directory(
                          train_dir, 
                          target_size = (img_width,img_height),
                          batch_size = batch_s,
                          class_mode = 'categorical',
                          subset = 'training'
                          )


In [0]:
validation_generator = train_datagen.flow_from_directory(
                          train_dir, 
                          target_size = (img_width,img_height),
                          batch_size = batch_s,
                          class_mode = 'categorical',
                          subset = 'validation'
                          )
       

#Create and Train Model


In [0]:
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation = 'relu', kernel_initializer = initializers.RandomNormal(stddev = 0.01), bias_initializer = initializers.Zeros()))
model.add(layers.Dense(5, activation = 'sigmoid', kernel_initializer = initializers.RandomNormal(stddev = 0.01), bias_initializer = initializers.Zeros()))
for layer in base_model.layers:
    layer.trainable = False

model.summary()


In [0]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
history = model.fit_generator(
                    train_generator,
                    epochs = epochs,
                    validation_data = validation_generator
                    )


In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [0]:
model.save("/content/gdrive/My Drive/Keras Models/Xception_5-1.h5")


In [0]:
# image folder
folder_path = 'Testbilder/'
# path to model
model_path = '/path/to/saved/model.h5'

# load all images into a list
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images)
print(classes)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [0]:
import json
def Vorhersage(Artikelfoto, Speicherort_KI_Modell, Name_KI_Modell):

    model = models.load_model(Speicherort_KI_Modell + Name_KI_Modell)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    Artikelfoto = image.load_img(Artikelfoto, target_size=(img_width, img_height))
    Artikelfoto = image.img_to_array(Artikelfoto)
    Artikelfoto = np.expand_dims(Artikelfoto, axis=0)
    prediction = model.predict(Artikelfoto)
    prediction = prediction.flatten()
    artikelnummer = 'Art' + str(np.argmax(prediction)+1)
    wahrscheinlichkeit = str(prediction[np.argmax(prediction)])

    response = json.dumps({"Artikelnummer":artikelnummer, "Wahrscheinlichkeit":wahrscheinlichkeit})

    return response

In [0]:
Vorhersage(base_dir + Abfragebild, Speicherort_KI_Modell, '/Xception_5-1.h5')

'{"Artikelnummer": "Art1", "Wahrscheinlichkeit": "0.0"}'